In [1]:
import pretty_midi
import symusic
import mido
import numpy as np
import joblib
import glob
import matplotlib.pyplot as plt
# %matplotlib inline
# import seaborn as sns
# sns.set_style('white')
# sns.set_context('notebook', font_scale=1.5)
import matplotlib.gridspec
import collections
import os
from pathlib import Path
# plotting.py contains utility functions for making nice histogram plots
# import util.plotting as plotting
from util.play_midi import play_midi
from statistics import mean, stdev, variance
import random
import miditok
import pandas as pd
from tqdm import tqdm 
import csv

from huggingface_hub import notebook_login
import ipywidgets

In [2]:
if "lofi_radio" in os.getcwd().split(os.sep)[-1].lower():
    os.chdir("..")

# if os.chdir("..")
print(os.getcwd())

midi_files = [str(file) for file in Path("pretraining_dataset/").glob("**/*.mid")]
midi_file2 = [str(file) for file in Path("lofi_radio/data/jazz_midi").glob("*.mid")]
print(len(midi_files))

/mnt/sdb1/big_documents/uni_surrey/Project
138174


In [9]:
notebook_login()

In [3]:
tokenizer = miditok.REMI.from_pretrained("theglassofwater/remi_12500")

/usr/local/lib/python3.10/dist-packages/miditok/midi_tokenizer.py:3252: UserWarning: The special token PAD_None is present twice in your configuration. Skipping its duplicated occurrence.
  self.config = TokenizerConfig()
/usr/local/lib/python3.10/dist-packages/miditok/classes.py:702: UserWarning: The special token PAD_None is present twice in your configuration. Skipping its duplicated occurrence.
  return cls(**input_dict, **kwargs)


In [4]:
seq_len = 100_000
full_dataset = miditok.pytorch_data.DatasetMIDI(
    files_paths=midi_files[:1000],
    max_seq_len=seq_len+1,  # max_seq_len = start + seq_len + end
    tokenizer=tokenizer,
    bos_token_id=tokenizer["BOS_None"],
    eos_token_id=tokenizer["EOS_None"],
)

In [5]:
def tokenize_file(file):
    try:
        tokenized_file = file["input_ids"]
        return tokenized_file
    except:
        return None

tokenized_dataset = joblib.Parallel(n_jobs=12, verbose=1)(
    joblib.delayed(tokenize_file)(midi_file)
    for midi_file in full_dataset)

sections_n = 20
sections_size = int(len(tokenized_dataset)/20)
for i in range(sections_n):
    hf_csv = pd.DataFrame({"input_id":[[int(i) for i in s] for s in tokenized_dataset[int(i*sections_size):int((i+1)*sections_size)] if s is not None]})
    hf_csv.to_csv(f"tokenized_datasets/tokenized_dataset_{i}.csv")

hf_csv = pd.DataFrame({"input_id":[[int(i) for i in s] for s in tokenized_dataset if s is not None]})
hf_csv.to_csv("tokenized_dataset.csv")


[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    7.3s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:   22.8s
[Parallel(n_jobs=12)]: Done 426 tasks      | elapsed:   49.3s
[Parallel(n_jobs=12)]: Done 776 tasks      | elapsed:  1.5min
[Parallel(n_jobs=12)]: Done 1000 out of 1000 | elapsed:  1.8min finished


In [7]:
import csv
with open('tokenized_pretraining_datset.csv', 'a', newline='') as f:  # 'a' opens the file in append mode
    writer = csv.writer(f)
    for result in tokenized_dataset:
        if result is not None:
            writer.writerow(int(result))

Error: iterable expected, not NoneType

In [25]:
sections_n = 20
sections_size = int(len(tokenized_dataset)/20)
for i in range(sections_n):
    hf_csv = pd.DataFrame({"input_id":[[int(i) for i in s] for s in tokenized_dataset[int(i*sections_size):int((i+1)*sections_size)] if s is not None]})
    hf_csv.to_csv(f"tokenized_datasets/tokenized_dataset_{i}.csv")
# hf_csv

In [23]:
hf_read_csv = pd.read_csv("tokenized_dataset0.csv")
hf_read_csv

,Unnamed: 0,input_id
0,0,"[1, 4, 416, 594, 11103, 654, 450, 594, 1819, 1..."
1,1,"[1, 416, 3790, 558, 640, 558, 682, 7110, 893, ..."
2,2,"[1, 4, 416, 2460, 2030, 2460, 2391, 2460, 2295..."
3,3,"[1, 4, 4, 1633, 454, 993, 454, 950, 443, 1141,..."
4,4,"[1, 416, 923, 4102, 61, 1416, 193, 658, 3744, ..."
...,...,...
195,195,"[1, 1176, 3747, 1304, 194, 2171, 7443, 2154, 7..."
196,196,"[1, 4, 969, 1221, 769, 1089, 213, 1352, 1255, ..."
197,197,"[1, 4, 206, 4097, 985, 211, 4529, 872, 217, 25..."
198,198,"[1, 4, 4, 416, 6008, 809, 1041, 583, 4537, 558..."


In [15]:
print(len(tokenized_dataset))
print(len([s for s in tokenized_dataset if s is None]))

1000
2


In [ ]:
hf_csv = pd.DataFrame({"input_id":tokenized_dataset})
hf_csv.to_csv("tokenized_dataset.csv")

In [ ]:
from datasets import load_dataset

hf_dataset = load_dataset("pretraining_tokenized_dataset", data_dir="Users/joeac/Documents/Uni/Y3/Project/tokenized_dataset.csv"))
hf_dataset.push_to_hub("theglassofwater/pretraining_tokenized_dataset")